# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [3]:
import pandas as pd
import pymongo

In [4]:
client = pymongo.MongoClient("mongodb://192.168.1.10:27017/")
database = client['exercices']
collection = database['youtube']

In [5]:
df_category = pd.read_json('./data/US_category_id.json')

In [6]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube = df_youtube.rename(columns={"category_id":"category"})

for i in range(len(df_youtube)):
    for j in range(len(df_category)):
        if df_youtube['category'][i] == int(df_category['items'][j]['id']):
            df_youtube.loc[i, 'category'] = df_category['items'][j]['snippet']['title']
            break

In [7]:
df_youtube.head()

,video_id,title,channel_title,category,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,Entertainment,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,Science & Technology,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09
2,cLdxuaxaQwc,My Response,PewDiePie,People & Blogs,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,Science & Technology,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,Comedy,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09


## Question 0

### Netoyer les données

In [13]:
df_youtube.isna().sum()

In [9]:
df_youtube = df_youtube.drop_duplicates(subset=['video_id'], keep='last')

In [10]:
 df_youtube = df_youtube.rename(columns={"video_id":"_id"})

In [11]:
df_youtube['tags'] = df_youtube['tags'].str.split('|')

### Importer les données

In [14]:
collection.insert_many(df_youtube.to_dict('records'))
cursor = collection.find()

In [24]:
next(cursor)

{'_id': 'U2CqZNd6rgM',
 'title': 'Welcome to JURASSIC FALLS!',
 'channel_title': 'Brave Wilderness',
 'category': 'Pets & Animals',
 'tags': ['adventure',
  'adventurous',
  'animals',
  'brave',
  'brave wilderness',
  'breaking trail',
  'coyote',
  'coyote peterson',
  'coyotes backyard',
  'dragon tails',
  'peterson',
  'trail',
  'wild',
  'wildlife',
  'dinosaurs',
  'jurassic park',
  'jurassic world',
  't rex',
  'jurassic',
  'dino',
  'living dinosaur',
  'the lost world',
  'jurassic falls',
  'welcome to jurassic falls',
  'devinsupertramp',
  'manawaiopuna falls',
  'helicopter',
  'na pali coast',
  'island helicopters',
  'waimea canyon',
  'helicopter tour',
  'jurassic park 4',
  'jurassic park movie',
  'jurassic park trailer',
  'adrenaline'],
 'views': 2384595,
 'likes': 50009,
 'dislikes': 1918,
 'comment_total': 6584,
 'thumbnail_link': 'https://i.ytimg.com/vi/U2CqZNd6rgM/default.jpg',
 'date': 13.09}

## Question 1  

In [25]:
cur = collection.find({"channel_title":"Apple"})
list(cur)

[{'_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category': 'Science & Technology',
  'tags': ['Apple',
   'iPhone 10',
   'iPhone Ten',
   'iPhone',
   'Portrait Lighting',
   'A11 Bionic',
   'augmented reality',
   'emoji',
   'animoji',
   'Face ID',
   'Apple Pay',
   'camera',
   'smartphone'],
  'views': 19707391,
  'likes': 381919,
  'dislikes': 69465,
  'comment_total': 0,
  'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
  'date': 17.09},
 {'_id': 'A9k88sMyiJM',
  'title': 'Apple Watch Series 3 + Apple Music — Roll — Apple',
  'channel_title': 'Apple',
  'category': 'Science & Technology',
  'tags': ['Apple',
   'airpods',
   'apple watch',
   'apple watch series 3',
   'apple watch cellular',
   'apple watch 4g',
   'apple music',
   '40 million songs on your wrist',
   'apple skateboard ad',
   'apple train station ad',
   'apple roll ad',
   'apple watch skateboard ad',
   'apple watch train st

## Question 2

In [26]:
cur = collection.aggregate([{"$group" : {"_id" : "$category", "count" : {"$sum" : 1}}}])
len(list(cur))

16

## Question 3

In [28]:
# already done

## Question 4

In [27]:
cur = collection.find().sort([("views", -1)])
for i in range(5):
    print(next(cur))
    print()

{'_id': 'MBdVXkSdhwU', 'title': "BTS (방탄소년단) 'DNA' Official MV", 'channel_title': 'ibighit', 'category': 'Music', 'tags': ['BIGHIT', '빅히트', '방탄소년단', 'BTS', 'BANGTAN', '방탄'], 'views': 41500672, 'likes': 2010366, 'dislikes': 78076, 'comment_total': 736179, 'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg', 'date': 22.09}

{'_id': 'tt2k8PGm-TI', 'title': 'ZAYN - Dusk Till Dawn ft. Sia', 'channel_title': 'ZaynVEVO', 'category': 'Music', 'tags': ['Dusk Till Dawn', 'Pop', 'RCA Records Label', 'ZAYN feat. Sia'], 'views': 36323498, 'likes': 1431683, 'dislikes': 28049, 'comment_total': 100661, 'thumbnail_link': 'https://i.ytimg.com/vi/tt2k8PGm-TI/default.jpg', 'date': 13.09}

{'_id': 'LunHybOKIjU', 'title': 'Eminem Rips Donald Trump In BET Hip Hop Awards Freestyle Cypher', 'channel_title': 'BETNetworks', 'category': 'Entertainment', 'tags': ['BET', 'BET Networks', 'BET Music', 'Black Entertainment Television', 'eminem', 'freestyle', 'full', 'hip hop awards', 'hip hop awards 201

## Question 5

In [29]:
cur = collection.aggregate([{"$group" : {"_id" : "$category", "average_view_by_category" : {"$avg" : "$views"}}}])
list(cur)

[{'_id': 'Music', 'average_view_by_category': 1442240.943283582},
 {'_id': 'Pets & Animals', 'average_view_by_category': 633906.4210526316},
 {'_id': 'Film & Animation', 'average_view_by_category': 966547.2871287129},
 {'_id': 'Gaming', 'average_view_by_category': 777213.0357142857},
 {'_id': 'News & Politics', 'average_view_by_category': 533575.1262626263},
 {'_id': 'Howto & Style', 'average_view_by_category': 556002.6679389313},
 {'_id': 'Nonprofits & Activism', 'average_view_by_category': 681178.2},
 {'_id': 'Comedy', 'average_view_by_category': 1317200.7289719626},
 {'_id': 'Shows', 'average_view_by_category': 8726.0},
 {'_id': 'Entertainment', 'average_view_by_category': 1059427.7735849055},
 {'_id': 'Education', 'average_view_by_category': 550585.2659574468},
 {'_id': 'Science & Technology',
  'average_view_by_category': 861563.6158940397},
 {'_id': 'People & Blogs', 'average_view_by_category': 892992.28515625},
 {'_id': 'Travel & Events', 'average_view_by_category': 426492.22222

## Question 6 

In [30]:
cur = collection.aggregate([{"$group" : {"_id" : "$channel_title","average_likes_by_channel" : {"$avg" : "$likes"}}}, {"$sort" : {"average_likes_by_channel" : -1 }}])
list(cur)[:10]

[{'_id': 'ibighit', 'average_likes_by_channel': 1533457.0},
 {'_id': 'ZaynVEVO', 'average_likes_by_channel': 1431683.0},
 {'_id': 'BETNetworks', 'average_likes_by_channel': 982842.0},
 {'_id': 'melanie martinez', 'average_likes_by_channel': 921577.0},
 {'_id': 'jypentertainment', 'average_likes_by_channel': 793387.0},
 {'_id': 'Logan Paul Vlogs', 'average_likes_by_channel': 724478.5},
 {'_id': 'Wengie', 'average_likes_by_channel': 656526.0},
 {'_id': 'Alesso', 'average_likes_by_channel': 618436.0},
 {'_id': 'shakiraVEVO', 'average_likes_by_channel': 578266.0},
 {'_id': 'TaylorSwiftVEVO', 'average_likes_by_channel': 508015.0}]